### Título
Fernando A. Correia dos S. Junior<br>
João Vitor Chaves de Oliveira

In [1]:
import networkx as nx
import json
import matplotlib.pyplot as plt
import random
import re
%matplotlib inline

In [7]:
class Graph():
    def __init__(self, path, graph_name="grafo-normalizado"): #construtor
        self.graph_name = graph_name
        self.GraphJSON = json.load(open(path)) 
        self.G = nx.DiGraph() # Gera um grafo direcionado vazio


    def generateGraph1(self): #Esta é a função que gera o grafo. A outra está errada
        keys = self.GraphJSON.keys()
        for i in keys:
            values = self.GraphJSON[i]
            for j in values:
                self.G.add_edge(i,j)
        
    def getNumberOfEdges(self):
        return self.G.number_of_edges()

    def getNumberOfVertex(self):
        return self.G.number_of_nodes()
    
    def saveGraph(self): ## SALVA O GRAFO. Obs: Verificar a estrutura de diretórios
        nx.write_graphml(self.G, path='%s.graphml' % self.graph_name)
        
    def centrality(self): ##Retorna centralidade de grau
        centralities = nx.algorithms.degree_centrality(self.G)
        print("Centralidade media: ",nx.density(self.G))
        return centralities
        
    def showGraph(self): #plota o grafo mas não fica tão legal 
        nx.draw_networkx(self.G)
        plt.show()

In [8]:
G = Graph('normalizado_citation_map_aborto_lista_de_adj.json') #COnstrutor passa o JSON
G.generateGraph1() #Gera o grafo a partir do JSON
G.saveGraph() # SALVA O ARQUIVO, EU ploto na ferramenta cytoscape

In [5]:
centralities = G.centrality()

('Centralidade media: ', 0.0011553409722746108)


In [6]:
print len(set(centralities.values()))
centralities = [(k, v) for k, v in centralities.items()]
centralities.sort(key=lambda x: x[1])
centralities[:10]

41


[(u'ao 16 stf', 0.0008361204013377926),
 (u'rhc 32490 stj', 0.0008361204013377926),
 (u'rhc 82288 stf', 0.0008361204013377926),
 (u'hc 109133 stf', 0.0008361204013377926),
 (u're 348827 stf', 0.0008361204013377926),
 (u'ext 785 stf', 0.0008361204013377926),
 (u'adc 12 stf', 0.0008361204013377926),
 (u're 596682 stf', 0.0008361204013377926),
 (u'hc 87921 stf', 0.0008361204013377926),
 (u'hc 93596 stf', 0.0008361204013377926)]

In [11]:
for ano in range(2005, 2016, 2):
    G = Graph('normalizado_citation_map_aborto_%s-%s.json' % (ano, ano + 1), "%s-%s" % (ano, ano + 1))
    G.generateGraph1()
    G.saveGraph()